In [14]:
import cvxpy as cp
import numpy as np

# Definir variáveis k, f
k, f = 1, 1
# Definir as matrizes A, B e C
A = np.array([[0, 0, 1, 0],
              [0, 0, 0, 1],
              [-k, k, -f, f],
              [k, -k, f, -f]])

B1 = np.array([[0], [0], [0], [1]])
B2 = np.array([[0], [0], [0], [1]])

# As matrizes C e D precisam ter 4 linhas para serem compatíveis com as
# variáveis W (4x4) na LMI.
C = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]])

D = np.array([[0], [0], [1], [0]])

# --- CORREÇÃO: Definir dimensões de forma clara ---
# n: dimensão do estado
# q1: dimensão da saída (linhas de C)
# m1: dimensão da perturbação (colunas de B1)
n = A.shape[0]
q1 = C.shape[0]
m1 = B1.shape[1]

# Definir as variáveis de otimização
W = cp.Variable((n, n), symmetric=True)
Z = cp.Variable((1, n)) # Assumindo 1 entrada de controle
mu = cp.Variable()

# --- CORREÇÃO: Blocos da LMI com as dimensões corretas ---
M11 = A@W + W@A.T + B2@Z + Z.T@B2.T
M12 = W@C.T + Z.T@D.T
M13 = B1
M21 = C@W + D@Z
# M22 deve ter dimensão (q1, q1)
M22 = -np.eye(q1)
# M23 deve ter dimensão (q1, m1)
M23 = np.zeros((q1, m1))
M31 = B1.T
# M32 deve ter dimensão (m1, q1)
M32 = np.zeros((m1, q1))
# M33 deve ter dimensão (m1, m1)
M33 = -mu * np.eye(m1)

# Construir a matriz de blocos LMI
# A LMI resultante terá dimensão (n + q1 + m1) x (n + q1 + m1) = 9x9
LMI = cp.bmat([
    [M11, M12, M13],
    [M21, M22, M23],
    [M31, M32, M33]
])

# Definir a função objetivo: min (mu)
objective = cp.Minimize(mu)

# Definir as restrições
constraints = [LMI << 0]

# Criar e resolver o problema de otimização
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCS) # É uma boa prática especificar o solver

# Verificar o status da solução
if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print("Status:", problem.status, "\n")
    # --- CORREÇÃO: O objetivo é 'mu', não Tr(X) ---
    print("Valor ótimo de mu:")
    print(problem.value)
    print("\nMatriz W:")
    print(W.value)
    print("\nVetor Z:")
    print(Z.value)
else:
    print("O problema não tem uma solução ótima. Status:", problem.status)



Status: optimal 

Valor ótimo de mu:
1.000015846550934

Matriz W:
[[ 0.40978462 -0.42745248 -0.27010701  0.26990252]
 [-0.42745248  0.44378728  0.27940851 -0.27962951]
 [-0.27010701  0.27940851  1.45009541 -1.44972486]
 [ 0.26990252 -0.27962951 -1.44972486  1.44935317]]

Vetor Z:
[[-0.43239896  0.42081264  2.13051775 -3.13010303]]
